In [15]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
openai_client = OpenAI(api_key=os.environ['API_KEY'])

# Tavily APIクライアントを生成
tavily_client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]


# Tavilyで検索を実行し、results部分をJSON文字列にして返す関数
def get_search_result(question):
    response = tavily_client.search(question)
    return json.dumps({"result": response["results"]})


# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, messages):
    work_messages = list(messages)
    work_messages.append(response.choices[0].message)

    for tool in response.choices[0].message.tool_calls:
        function_response = execute_tool_call(tool)

        work_messages.append(
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            }
        )

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=work_messages,
    )
    return response_after_tool_call


# ツール呼び出しを実行する関数
def execute_tool_call(tool):
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)
    return function_response


# ユーザーからの質問を処理する関数
def process_response(messages, tools):
    response = ask_question(messages, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, messages)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()


# 言語モデルへの質問を行う関数
def ask_question(messages, tools):
    response = openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    return response


# チャットボットへの組み込み
tools = define_tools()

messages=[]

try:
    while(True):
        # ユーザーからの質問を受付
        question = input("メッセージを入力:")
        # 質問が入力されなければ終了
        if question.strip()=="":
            break
        display(f"質問:{question}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": question.strip()})
        # やりとりが8を超えたら古いメッセージから削除
        if len(messages) > 8:
            del_message = messages.pop(0)

        # 言語モデルに質問
        response_message = process_response(messages, tools)

        # メッセージに言語モデルからの回答を追加
        print(response_message, flush=True)
        messages.append({"role": "assistant", "content": response_message})

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")

'質問:仙台駅で牛たん弁当が買える店を 営業時間/場所/URL 付きで教えて'

仙台駅で牛たん弁当が購入できる店舗について、以下の情報を提供します。

1. **牛たん通り店 (Gyutandori)**
   - **営業時間**: 10:00〜21:30 (L.O. 21:00)
   - **場所**: JR仙台駅構内、3F
   - **URL**: [公式サイト](https://www.tanya-zenjirou.jp/restaurant/gyutandori.html)

2. **牛たん本舗 仙台駅店**
   - **営業時間**: 11:00〜22:00 (L.O. 21:00)
   - **場所**: tekute仙台駅内
   - **URL**: [グルナビ情報](https://r.gnavi.co.jp/t274204/)

3. **レストランれんが (Restaurant Reng)**
   - **営業時間**: 11:00〜21:30 (L.O. 20:30)
   - **場所**: 仙台駅近隣
   - **URL**: [公式サイト](https://www.dategyu.jp/restaurant/)

これらの店舗で美味しい牛たん弁当を楽しむことができますので、ぜひ訪れてみてください。
処理を中断しました。
---ご利用ありがとうございました！---
